In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
#from pretrainedModel import pretrainedModel
from tensorflow import keras
from PIL import Image
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import time
import warnings
from sklearn.cluster import KMeans
import sys
import os

from copy import deepcopy

In [2]:
import random
import torch.nn as nn
import numpy as np
import torch
import math
import torch.optim as optim
from torch.utils.data import DataLoader

root = '../'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def get_dataloader(X_train, y_train, one_hot=True, batchSize = 100):
    y_train = torch.tensor(y_train, dtype=torch.long, device=device)

    if one_hot:
        y_train = torch.nn.functional.one_hot(y_train)

    train_data = []
    for i in range(len(X_train)):
        train_data.append([X_train[i], y_train[i]])
    dataloader = DataLoader(train_data, batch_size=batchSize, shuffle=True)


    return dataloader

In [5]:
class TripletLoss(torch.nn.Module):
    def __init__(self):
        super(TripletLoss, self).__init__()

    def forward(self, u, y, ALPHA=1):
        #LAMBDA = 1
        #ALPHA  = 1

        inner_product = torch.cdist(u, u, p=2) 
        s = y @ y.t() > 0           # A matrix that show if the two idexes are the same or not

        loss1 = torch.tensor(0.0, requires_grad=True) + 0
        for row in range(s.shape[0]):
            # if has positive pairs and negative pairs
            if s[row].sum() != 0 and (~s[row]).sum() != 0:
                
                theta_negative = inner_product[row][s[row] == 0]
                
                theta_positive = inner_product[row][s[row] == 1]
                theta_positive = theta_positive[theta_positive != 0] # remove the anchor

                for p in theta_positive: 
                    n_i = torch.logical_and( (p < theta_negative), (theta_negative < p + ALPHA) )
                    
                    if sum(n_i) != 0:
                        n = torch.min( theta_negative[n_i] )

                        loss1 += (p - n + ALPHA).clamp(min=0)
        

        return loss1

In [6]:
def earlyStop(LossList, n = 10):
    bestVal = min(LossList)

    bestVal_i = LossList.index(bestVal)

    if bestVal_i < len(LossList) - n: return True



def HPO(HP, X_train, y_train, do_one_hot=True, earlyStop_num=20):
    data = {}

    ALPHA=  HP["alpha"]
    lr=     HP["lr"]
    weight_decay= HP["wd"]
    bits = HP["bits"]


    model = nn.Sequential(  nn.Linear(4096,256),
                            nn.ReLU(),
                            nn.Linear(256, bits),
                            nn.Sigmoid()
                            )
    model.to(device)

    criterion = TripletLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=lr , weight_decay=weight_decay)

    dataloader = get_dataloader(X_train, y_train, one_hot=do_one_hot, batchSize = 50)
    historical_lostList = []
    for i in range(1500):
        loss_list = []
        for j,batch  in enumerate(dataloader):
            X_batch = batch[0]
            y_batch = batch[1]

            optimizer.zero_grad()

            u = model(X_batch)
            loss = criterion(u, y_batch.float(), ALPHA=ALPHA)
            loss.backward()
            optimizer.step()

            loss_list.append( float(loss) )
        
        
        mean_loss = sum(loss_list) / len(loss_list)
        if (i % 10 == 1) or True:
            print(i, mean_loss)
        historical_lostList.append(mean_loss)

        if earlyStop(historical_lostList, n = earlyStop_num): 
            print(i, mean_loss)
            print("Early Stop!!!")
            data["earlyStop"] = True
            break
    
    return model

In [7]:
Cifar = [
    {"alpha": 3, "lr":0.0001, "wd": 0.000010, "bits" : 12},
    {"alpha": 5, "lr":0.0001, "wd": 0.000100, "bits" : 48},
    {"alpha": 3, "lr":0.0001, "wd": 0.000001, "bits" : 32},
    {"alpha": 3, "lr":0.0001, "wd": 0.000100, "bits" : 24},
       ]

Nus = [
    {"alpha": 5, "lr":0.0001, "wd": 0.00001, "bits" : 32},
    {"alpha": 3, "lr":0.0001, "wd": 0.00001, "bits" : 12},
    {"alpha": 3, "lr":0.0001, "wd": 0.00010, "bits" : 48},
    {"alpha": 5, "lr":0.0001, "wd": 0.00001, "bits" : 24},
       ]

ImgNet = [
    {"alpha": 5, "lr":0.00010, "wd": 0.00001, "bits" : 32},
    {"alpha": 1, "lr":0.00001, "wd": 0.00010, "bits" : 48},
    {"alpha": 5, "lr":0.00010, "wd": 0.00001, "bits" : 12},
    {"alpha": 5, "lr":0.00010, "wd": 0.00001, "bits" : 24},
       ]




hpo_dic ={
    12: {"Cifar":   {"alpha": 3, "lr":0.0001, "wd": 0.000010, "bits" : 12}, 
         "Nus_Wide":{"alpha": 3, "lr":0.0001, "wd": 0.00001, "bits" : 12}, 
         "Imagenet":{"alpha": 5, "lr":0.00010, "wd": 0.00001, "bits" : 12}},

    24: {"Cifar":   {"alpha": 3, "lr":0.0001, "wd": 0.000100, "bits" : 24}, 
         "Nus_Wide":{"alpha": 5, "lr":0.0001, "wd": 0.00001, "bits" : 24}, 
         "Imagenet":{"alpha": 5, "lr":0.00010, "wd": 0.00001, "bits" : 24}},

    32: {"Cifar":   {"alpha": 3, "lr":0.0001, "wd": 0.000001, "bits" : 32}, 
         "Nus_Wide":{"alpha": 5, "lr":0.0001, "wd": 0.00001, "bits" : 32}, 
         "Imagenet":{"alpha": 5, "lr":0.00010, "wd": 0.00001, "bits" : 32}},

    48: {"Cifar":   {"alpha": 5, "lr":0.0001, "wd": 0.000100, "bits" : 48}, 
         "Nus_Wide":{"alpha": 3, "lr":0.0001, "wd": 0.00010, "bits" : 48}, 
         "Imagenet":{"alpha": 1, "lr":0.00001, "wd": 0.00010, "bits" : 48}},
        }



hp = hpo_dic[12]["Cifar"]

X_train = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/X_hpo_Cifar.npy" ) )
y_train = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/y_hpo_CIfar.npy" ) )

model = HPO(hp, X_train, y_train, earlyStop_num=1) # ! SET EARLY STOP "earlyStop_num" TO 20 (hpo er lavet på 10, så det vil nok også gå hvis vi har travlt)

C:\Users\thors\AppData\Local\Temp\ipykernel_6200\4162365391.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long, device=device)


0 731.7476824951171
1 701.4269555664063
2 705.2783355712891
2 705.2783355712891
Early Stop!!!


In [8]:
X_val = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/X_val_Cifar.npy" ) )

model(X_val[22])

tensor([9.7070e-01, 1.3643e-01, 4.1810e-02, 8.3139e-01, 2.9377e-01, 6.1342e-04,
        1.0688e-01, 9.1936e-01, 9.0135e-01, 2.7574e-01, 9.8846e-01, 9.8929e-01],
       grad_fn=<SigmoidBackward0>)